In [284]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.stats import entropy as en
from math import log,e
import sklearn
from IPython.display import Image
from IPython.core.display import HTML

%matplotlib inline


# Setul de date initial

In [285]:
df= pd.read_csv(r"C:\Users\student\DataspellProjects\Running\datasetInitial.csv")
df

,date,time,username,wrist,activity,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z
0,2017-6-30,13:51:15:847724020,viktor,0,0,0.2650,-0.7814,-0.0076,-0.0590,0.0325,-2.9296
1,2017-6-30,13:51:16:246945023,viktor,0,0,0.6722,-1.1233,-0.2344,-0.1757,0.0208,0.1269
2,2017-6-30,13:51:16:446233987,viktor,0,0,0.4399,-1.4817,0.0722,-0.9105,0.1063,-2.4367
3,2017-6-30,13:51:16:646117985,viktor,0,0,0.3031,-0.8125,0.0888,0.1199,-0.4099,-2.9336
4,2017-6-30,13:51:16:846738994,viktor,0,0,0.4814,-0.9312,0.0359,0.0527,0.4379,2.4922
...,...,...,...,...,...,...,...,...,...,...,...
88583,2017-7-9,20:9:15:317911028,viktor,0,0,0.3084,-0.8376,-0.1327,0.4823,2.0124,0.6048
88584,2017-7-9,20:9:15:517889022,viktor,0,0,0.4977,-1.0027,-0.4397,0.1022,-1.2565,-0.0761
88585,2017-7-9,20:9:15:717828989,viktor,0,0,0.4587,-1.1780,-0.2827,-1.4500,-0.2792,-1.2616
88586,2017-7-9,20:9:15:917932987,viktor,0,0,0.2590,-0.8582,-0.0759,-1.5165,0.4560,-1.7755


Setul de date este preluat de pe Kaggle : https://www.kaggle.com/datasets/vmalyi/run-or-walk/data

Datele au fost înregistrate de către Viktor Malyi pe un telefon mobil de tipul "iPhone 5c" prin o aplicație dezvoltată în mod special pentru a servi la colectarea datelor.

În setul de date se observă prezența a 11 coloane :
    Coloanele "date" și "time"  aduc referința momentului în care datele au fost înregistrate.
    Coloana "username" prezintă utilizatorul care a înregistrat datele.
    Coloana "wrist" indică încheietura mâinii pe care era atasat device-ul în momentul înregistrării datelor( 0 : stânga, 1 : dreapta)
    Coloana "activity" indică dacă datele au fost înregistrate în timpul alergării sau al mersului  ( 0 : mers, 1 : alergat)
    Coloanele "acceleration_x", "acceleration_y", "acceleration_z" ce reprezintă schimbarea accelerometrului în cele 3 direcții.
 
Datele sunt distribuite uniform, majoritatea aflându-se în limitele normale iar cele care sunt aflate în extreme nu pot fi considerate eronate din cauza modului de colectare a datelor.

## Date neimportante

Coloanele "username", "date" și "time" nu sunt necesare acestei aplicații, astfel nu vor fi considerate în antrenarea modelelor.

In [286]:
# df.corr(numeric_only=True)
df=df.drop('username',axis=1)
df=df.drop('time',axis=1)
df=df.drop('date',axis=1)
corr= df.select_dtypes('number').corr()



cmap = sns.diverging_palette(100, 5, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
            ]

corr.style.background_gradient(cmap, axis=1) \
    .format(precision=3) \
    .set_properties(**{'max-width': '120px', 'font-size': '10pt'}) \
    .set_caption("Hover to magify") \
    .set_table_styles(magnify())




#sns.heatmap(corr)

,wrist,activity,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z
wrist,1.000,-0.113,-0.610,0.087,0.324,-0.022,-0.068,0.009
activity,-0.113,1.000,-0.018,0.640,-0.192,0.041,0.012,-0.008
acceleration_x,-0.610,-0.018,1.000,-0.265,-0.552,-0.022,-0.004,-0.061
acceleration_y,0.087,0.640,-0.265,1.000,0.106,0.011,0.072,-0.023
acceleration_z,0.324,-0.192,-0.552,0.106,1.000,0.035,-0.021,0.050
gyro_x,-0.022,0.041,-0.022,0.011,0.035,1.000,0.094,0.318
gyro_y,-0.068,0.012,-0.004,0.072,-0.021,0.094,1.000,0.287
gyro_z,0.009,-0.008,-0.061,-0.023,0.050,0.318,0.287,1.000


# Corelații

În imaginea prezentă în continuare se observă axele de recoltare a datelor.
Această imagine este utilă în înțelegerea corelațiilor explicate mai jos.
    
   




 ![Imagine Axe Accelerometru](https://docs-assets.developer.apple.com/published/96e9d46b41/c9b606b2-9a52-487e-8385-e710ffa1ce5f.png)



Din imagine se poate explica de ce există o corelație mare între "wrist" și direcțiile de accelerare pe X și Y.
Telefonul a fost rotit pe încheietură astfel încât datele colectate de pe mâna stângă sunt tind să fie pozitive iar cele colectate de pe mâna sângă tind să fie negative. Trebuie menționat că nu este important semnul datelor, ci modulul diferenței lor față de 0.
"Acceleration_y" și "activity" prezintă o corelație ridicată, implicându-se creșterea accelerației atunci când "activity" este 1 ( 1 indicând alergat )
Din rotația telefonului pe încheietură cât și din încheietura pe care a fost ținută se explică și corelațiile între accelerațiile prezente în tabel. ("acceleration_x" cu "acceleartion_z"
Se observă că datele giroscopice nu au corelația ridicată în legătura cu restul datelor.

# Calcularea entropiei

In [287]:
def entropy1(labels, base=None):
    value,counts = np.unique(labels, return_counts=True)
    return en(counts, base=base)

def entropy2(labels, base=None):
    """ Computes entropy of label distribution. """

    n_labels = len(labels)

    if n_labels <= 1:
        return 0

    value,counts = np.unique(labels, return_counts=True)
    probs = counts / n_labels
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    ent = 0.

    # Compute entropy
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)

    return ent



def entropy3(attributes, dataset, targetAttr):
    freq = {}
    entropy = 0.0
    index = 0
    for item in attributes:
        if (targetAttr == item):
            break
        else:
            index = index + 1
    index = index - 1
    for item in dataset:
        if ((item[index]) in freq):
            # Increase the index
            freq[item[index]] += 1.0
        else:
            # Initialize it by setting it to 0
            freq[item[index]] = 1.0

    for freq in freq.values():
        entropy = entropy + (-freq / len(dataset)) * log(freq / len(dataset), 2)
    return entropy

max_entropy=np.log(df['acceleration_x'].count())


print(max_entropy)
print(entropy2(df['acceleration_x']))
print(entropy2(df['acceleration_y']))
print(entropy2(df['acceleration_z']))




11.391751687239736
9.936410209489402
9.796824629905597
9.347039399713736


Entropia maximă a unui set de valori poate fi calculată prin formula log(lugnime_set).
Au fost preluate diferite formule de calculare a entropiei din mediul online, însă după testare funcția "entropy2" a prezentat cele mai consistente rezultate.
Din valorile obținute se observă că datele prezente în această bază de date sunt bine împrăștiate, ceea ce va oferi algortmilor o varietate de cazuri și va antrena algorimtul în a generaliza, reducând riscul de supraînvățare.

# Impărțirea datelor

Separăm datele și le împărțim in setul de antrenament și setul de cros-validare

In [288]:
y= df['activity']
x= df.drop('activity',axis=1)


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.2)
X_train.shape, X_test.shape,Y_train.shape, Y_test.shape


((70870, 7), (17718, 7), (70870,), (17718,))

# Modele 

## Regresie liniară

**Antrenarea Modelului**

In [289]:
from sklearn.linear_model import LinearRegression

lr =LinearRegression()
lr.fit(X_train, Y_train)

LinearRegression()

**Aplicarea Modelului**

In [290]:
y_lr_train_pred=lr.predict(X_train)
y_lr_test_pred=lr.predict(X_test)



**Evaluarea Performanței Modelului**

In [291]:
from sklearn.metrics import mean_squared_error, r2_score
lr_train_mse=mean_squared_error(Y_train,y_lr_train_pred)
lr_train_r2=r2_score(Y_train,y_lr_train_pred)

lr_test_mse=mean_squared_error(Y_test,y_lr_test_pred)
lr_test_r2=r2_score(Y_test,y_lr_test_pred)



print("Valoarea MSE pe setul de antrenament este de: ",lr_train_mse)
print("Valoarea R2 pe setul de antrenament este de :",lr_train_r2)
print("Valoarea MSE pe setul de test este de: ",lr_test_mse)
print("Valoarea R2 pe setul de test este de :",lr_test_r2)

Valoarea MSE pe setul de antrenament este de:  0.12674977369353352
Valoarea R2 pe setul de antrenament este de : 0.49300069162738247
Valoarea MSE pe setul de test este de:  0.12758999750558653
Valoarea R2 pe setul de test este de : 0.4895825438646487


Din rezultatele obținute se observă că modelul ce implementează regresia liniară are un MSE ( mean squared error ) mic, ceea ce indică o performanță ridicată a modelului


# Regresie Logistică
**Antrenarea Modelului**

cum comparam algoritmii
xgboost -
random forest -
decision  trees -

In [292]:
from sklearn.linear_model import LogisticRegression

In [293]:
lor=LogisticRegression()
lor.fit(X_train, Y_train)

LogisticRegression()

**Aplicarea Modelului**

In [294]:
y_lor_train_pred=lor.predict(X_train)
y_lor_test_pred=lor.predict(X_test)

**Evaluarea Performanței Modelului**

In [295]:
from sklearn.metrics import accuracy_score

y_lor_predict=lor.predict(X_test)

accuracy_lor=accuracy_score(Y_test,y_lor_predict)
print("Acuratețea modelului este de : ",accuracy_lor*100," %")


Acuratețea modelului este de :  85.85619144372954  %


Din rezultatele obținute se observă că modelul implementat are o rată de succes mare. Acest model a fost ales pentru capacitatea sa de a clasifica rezultatele.

# XGBoost

**Antrenarea Modelului**

In [296]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=42,verbosity=0,silent=0)
xgb.fit(X_train,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

**Aplicarea Modelului**

In [297]:
y_xgb_predict = xgb.predict(X_test)
accuracy_xgb=accuracy_score(Y_test,y_xgb_predict)
print("Acuratețea modelului este de : ",accuracy_xgb*100," %")

Acuratețea modelului este de :  99.30014674342476  %


Din rezultatele obținute se observă cum modelul clasifică datele cu o acuratețe foarte ridicată, indicând că acest model este aproape perfect în a prezice dacă o persoană se află în fugă sau în mers.